# SUP ML 3 - PREDICT

# Librerías

In [1]:
! pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")

import pickle

# Carga modelo

* Cargar el modelo
* Obtener la lista de model features

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Cargar el modelo guardado desde el archivo
with open('/content/drive/MyDrive/DSC-07-dscesp-0624-Entregable 2- Kovylina, Talón, González, Benítez y Arderiu/data/modelo_best.pkl', 'rb') as archivo:
    modelo_best = pickle.load(archivo)


# Carga PREDICT dataset

In [5]:
# Cargar el DataFrame desde el archivo pickle
df_predecir = pd.read_pickle('/content/drive/MyDrive/DSC-07-dscesp-0624-Entregable 2- Kovylina, Talón, González, Benítez y Arderiu/data/df_clean_predecir.pkl')

In [6]:
# Procesamos el DataFrame df_predecir
X_predecir=df_predecir.drop(columns=["Customer_ID"])

# Guardamos variable "Customer_ID"
Customer_ID=df_predecir["Customer_ID"]


# ML Preprocessing

Se deben realizar las mismas transformaciones que se aplicaron en el preprocessing del train:
  * Eliminar mismas variables
  * Imputar mismos valores a nulos
  * Aplicar mismo encoding de categoricos
  * ...todo lo necesario para replicar las variables con las que el modelo se entrenó

NO SE DEBEN repetir:
  * Analisis visual de los datos
  * Analisis de correlaciones
  * Analisis de varianza
  * Calculo de métricas que varien con la distribución

Hecho en preprocessing

# Check model features

* Comprobar que tenemos en el dataset preprocesado todas las model features, de lo contrario no podremos hacer predict.
* Ordenar las variables en mismo orden que las model features

In [7]:
# Comprobamos las variables de modelo_best
feature_names = modelo_best.feature_names_
print(feature_names)

['crclscod_AA', 'dwllsize_A', 'dwllsize_Unknown', 'ethnic_N', 'prizm_social_one_S', 'new_cell_U', 'dualband_Y', 'HHstatin_C', 'HHstatin_Unknown', 'marital_M', 'marital_U', 'dwlltype_S', 'dwlltype_Unknown', 'ownrent_O', 'creditcd_Y', 'hnd_webcap_WCMB', 'rev', 'mou', 'totmrc', 'da', 'vceovr', 'roam', 'change_mou', 'change_rev', 'drop_vce', 'blck_vce', 'unan_vce', 'recv_vce', 'custcare', 'ccrndmou', 'threeway', 'mou_cvce', 'mou_rvce', 'owylis_vce', 'mouowylisv', 'iwylis_vce', 'mouiwylisv', 'peak_vce', 'mou_peav', 'opk_vce', 'mou_opkv', 'drop_blk', 'complete', 'callwait', 'months', 'uniqsubs', 'actvsubs', 'adjrev', 'adjmou', 'adjqty', 'avgrev', 'avgmou', 'avgqty', 'avg3qty', 'avg6rev', 'hnd_price', 'phones', 'models', 'lor', 'adults', 'income', 'numbcars', 'eqpdays']


# Rescaling

* Si se entrenó el modelo con un dataset estandarizado, estandarizar con mismo scaler.

El mejor modelo se entrenó sin Scaler ya que el mejor algoritmo es CatBoost y no requiere escalado

# PREDICT

* predict() y predict_proba()

In [8]:
# Obtener las predicciones
y_pred_clases = modelo_best.predict(X_predecir)

# Obtener las predicciones de probabilidad (probabilidad de clase 1)
y_pred_proba = modelo_best.predict_proba(X_predecir)[:, 1]

# Redondear las probabilidades a porcentajes con 1 decimal
y_pred_prob_porcentajes = np.round(y_pred_proba * 100, 1)

# Crear un DataFrame con Customer_ID, predicción y probabilidad
df_prediccion = pd.DataFrame({
    'Customer_ID': Customer_ID,
    'prediccion': y_pred_clases,
    '%_proba': y_pred_prob_porcentajes
})

# Mostrar el DataFrame
df_prediccion.head(50)



,Customer_ID,prediccion,%_proba
0,1090001,1,68.7
1,1090002,0,29.1
2,1090003,0,4.1
3,1090004,0,45.2
4,1090005,0,36.5
5,1090006,0,17.2
6,1090007,1,73.0
7,1090008,0,5.2
8,1090009,0,24.4
9,1090010,0,23.5


# Guarda predicciones

* Guardar las predicciones en data path. Cada fila debe estar etiquetada con el ID.

In [9]:
# Guardar el resultado en CSV si es necesario
df_prediccion.to_csv('/content/drive/MyDrive/DSC-07-dscesp-0624-Entregable 2- Kovylina, Talón, González, Benítez y Arderiu/data/predicciones.csv', index=False)